In [9]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import numpy as np

In [3]:
loader = PyPDFLoader('../../assets/pdf/2024_KB_부동산_보고서_최종.pdf')
pages = loader.load()
print('청크 수 : ', len(pages))

청크 수 :  84


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)
splits = text_splitter.split_documents(pages)
print('분할된 청크의 수 : ', len(splits))

분할된 청크의 수 :  135


In [10]:
chunk_lengths = [len(chunk.page_content) for chunk in splits]
print('청크의 최대 길이 : ', max(chunk_lengths))
print('청크의 최소 길이 : ', min(chunk_lengths))
print('평균 청크의 길이 : ', np.mean(chunk_lengths))

청크의 최대 길이 :  1000
청크의 최소 길이 :  56
평균 청크의 길이 :  674.9481481481481


In [11]:
vector_db = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(),
    persist_directory='../../assets/embedding')
print('문서의 수 : ', vector_db._collection.count())

문서의 수 :  135


In [13]:
vector_db = Chroma(
    persist_directory='../../assets/embedding/',
    embedding_function=OpenAIEmbeddings())
print('문서의 수 : ', vector_db._collection.count())

문서의 수 :  135


In [14]:
question = '수도권 주택 매매 전망'
top_three_docs = vector_db.similarity_search(query=question,
                                             k=2)
for i, doc in enumerate(top_three_docs):
    print(f'문서 : {i}')
    print(f'내용 : {doc.page_content}')
    print(f'메타데이터 : {doc.metadata}')

문서 : 0
내용 : 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가장 큰 원
인으로 볼 수 있다. 
또한 과거 전국적인 주택 거래 호황기와는 달리 서울 등 일부 지역을 중심으로 매수세가 형성되고, 
전세 사기를 포함한 보증금 미반환 리스크에 따라 비아파트에 대한 선호도가 낮아지면서 아파트로 수
요가 집중되었다. 거시경제 불확실성이 확대되면서 투자 수요 보다 실수요를 중심으로 매매 거래가 이
루어지는 등 다양한 원인이 복합적으로 작용하여 거래 침체를 유발하였다. 
이에 따라 꾸준한 주택 재고 증가에도 불구하고 주택 매매 거래회전율(전체 주택 재고 대비 주택 매
매거래 비율)은 최근 2년간 평균 3%를 하회하면서 지난 10년 전국 평균(6%) 대비 절반 수준에 머물
렀다. 전체 주택시장에 미치는 영향력이 큰 서울의 주택 거래회전율은 최근 2년 평균 1%로 전국 최저
치를 기록하였다. 주택시장이 안정적인 상황을 유지하기 위해서는 주택 구입·처분·이사 등 일련의 과정
이 반복되어야 한다. 거래 침체가 지속될 경우 주거 이전, 주택가격 왜곡과 연관 산업 침체, 지방자치단
체 조세 수입 감소 등 다양한 문제 발생으로 이어질 수 있어 주의가 필요하다.  
 
그림Ⅰ-14. 매수우위지수 및 매매가격전망지수 추이  그림Ⅰ-15. 전국 및 서울 주택 매매 거래회전율 추이 
 
 
 
주: 갭=매매가격전망지수-매수우위지수 
자료: KB국민은행  
자료: 부동산114 
 
 
■ 하반기 이후 거래 침체 현상이 다소 완화될 수 있으나 전반적인 회복세는 제한적일 것 
주택 매매 거래 회복을 위해서는 매도자와 매수자 간 희망가격 격차 축소와 매수세 회복이 중요한데, 
여전히 불확실한 주택가격 향방으로 주택시장에 대한 매도자와 매수자의 기대감이 달라 이러한 요인들
메타

In [19]:
question = '수도권 주택 매매 전망'
top_three_docs = vector_db.similarity_search_with_relevance_scores(query=question,
                                                                   k=3)
for i, doc in enumerate(top_three_docs, 1):
    print(f'문서 : {i}')
    print(f'유사도 : {doc[1]}')
    print(f'내용 : {doc[0].page_content}')
    print(f'메타데이터 : {doc[0].metadata}')

문서 : 1
유사도 : 0.8445251438043149
내용 : 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가장 큰 원
인으로 볼 수 있다. 
또한 과거 전국적인 주택 거래 호황기와는 달리 서울 등 일부 지역을 중심으로 매수세가 형성되고, 
전세 사기를 포함한 보증금 미반환 리스크에 따라 비아파트에 대한 선호도가 낮아지면서 아파트로 수
요가 집중되었다. 거시경제 불확실성이 확대되면서 투자 수요 보다 실수요를 중심으로 매매 거래가 이
루어지는 등 다양한 원인이 복합적으로 작용하여 거래 침체를 유발하였다. 
이에 따라 꾸준한 주택 재고 증가에도 불구하고 주택 매매 거래회전율(전체 주택 재고 대비 주택 매
매거래 비율)은 최근 2년간 평균 3%를 하회하면서 지난 10년 전국 평균(6%) 대비 절반 수준에 머물
렀다. 전체 주택시장에 미치는 영향력이 큰 서울의 주택 거래회전율은 최근 2년 평균 1%로 전국 최저
치를 기록하였다. 주택시장이 안정적인 상황을 유지하기 위해서는 주택 구입·처분·이사 등 일련의 과정
이 반복되어야 한다. 거래 침체가 지속될 경우 주거 이전, 주택가격 왜곡과 연관 산업 침체, 지방자치단
체 조세 수입 감소 등 다양한 문제 발생으로 이어질 수 있어 주의가 필요하다.  
 
그림Ⅰ-14. 매수우위지수 및 매매가격전망지수 추이  그림Ⅰ-15. 전국 및 서울 주택 매매 거래회전율 추이 
 
 
 
주: 갭=매매가격전망지수-매수우위지수 
자료: KB국민은행  
자료: 부동산114 
 
 
■ 하반기 이후 거래 침체 현상이 다소 완화될 수 있으나 전반적인 회복세는 제한적일 것 
주택 매매 거래 회복을 위해서는 매도자와 매수자 간 희망가격 격차 축소와 매수세 회복이 중요한데, 
여전히 불확실한 주택가격 향방으로 주택시장에 대한 매도자

In [21]:
from langchain_community.vectorstores import FAISS

In [25]:
faiss_db = FAISS.from_documents(
    embedding=OpenAIEmbeddings(),
    documents=splits)
faiss_db.save_local('../../assets/embedding/')

In [26]:
print('문서의 수 : ', faiss_db.index.ntotal)

문서의 수 :  135


In [27]:
faiss_db = FAISS.load_local(
    folder_path='../../assets/embedding/',
    embeddings=OpenAIEmbeddings(),
    allow_dangerous_deserialization=True)
print('문서의 수 : ', faiss_db.index.ntotal)

문서의 수 :  135


In [28]:
question = '수도권 주택 매매 전망'
docs = faiss_db.similarity_search(query=question)
for i, doc in enumerate(docs, 1):
    print(f'문서 : {i}')
    print(f'내용 : {doc.page_content}')
    print(f'메타데이터 : {doc.metadata}')

문서 : 1
내용 : 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가장 큰 원
인으로 볼 수 있다. 
또한 과거 전국적인 주택 거래 호황기와는 달리 서울 등 일부 지역을 중심으로 매수세가 형성되고, 
전세 사기를 포함한 보증금 미반환 리스크에 따라 비아파트에 대한 선호도가 낮아지면서 아파트로 수
요가 집중되었다. 거시경제 불확실성이 확대되면서 투자 수요 보다 실수요를 중심으로 매매 거래가 이
루어지는 등 다양한 원인이 복합적으로 작용하여 거래 침체를 유발하였다. 
이에 따라 꾸준한 주택 재고 증가에도 불구하고 주택 매매 거래회전율(전체 주택 재고 대비 주택 매
매거래 비율)은 최근 2년간 평균 3%를 하회하면서 지난 10년 전국 평균(6%) 대비 절반 수준에 머물
렀다. 전체 주택시장에 미치는 영향력이 큰 서울의 주택 거래회전율은 최근 2년 평균 1%로 전국 최저
치를 기록하였다. 주택시장이 안정적인 상황을 유지하기 위해서는 주택 구입·처분·이사 등 일련의 과정
이 반복되어야 한다. 거래 침체가 지속될 경우 주거 이전, 주택가격 왜곡과 연관 산업 침체, 지방자치단
체 조세 수입 감소 등 다양한 문제 발생으로 이어질 수 있어 주의가 필요하다.  
 
그림Ⅰ-14. 매수우위지수 및 매매가격전망지수 추이  그림Ⅰ-15. 전국 및 서울 주택 매매 거래회전율 추이 
 
 
 
주: 갭=매매가격전망지수-매수우위지수 
자료: KB국민은행  
자료: 부동산114 
 
 
■ 하반기 이후 거래 침체 현상이 다소 완화될 수 있으나 전반적인 회복세는 제한적일 것 
주택 매매 거래 회복을 위해서는 매도자와 매수자 간 희망가격 격차 축소와 매수세 회복이 중요한데, 
여전히 불확실한 주택가격 향방으로 주택시장에 대한 매도자와 매수자의 기대감이 달라 이러한 요인들
메타